In [11]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

!pip install llama-index

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [13]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [14]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")

In [15]:
llm = OpenAI(model="gpt-3.5-turbo")

In [17]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent

In [18]:
react_step_engine = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True)
agent = AgentRunner(react_step_engine)

In [19]:
# Start the agent interaction
print("Simple Chat Interaction:")
response = agent.chat("Hi")
print(response)

Simple Chat Interaction:
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow

ValueError: Reached max iterations.

In [20]:
# Handle a computational query
print("\nComputational Query Interaction:")
response = agent.chat("What is (121 * 3) + 42?")
print(response)


Computational Query Interaction:
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of (121 * 3) + 42 is 363.
The result of (121 * 3) + 42 is 363.


In [21]:
# Task-based Execution
print("\nStep-Wise Task Execution:")
task = agent.create_task("What is (121 * 3) + 42?")

# Step 1: Execute the first step of the task
step_output = agent.run_step(task.task_id)
print(f"Step 1 Output: {step_output.output.response}")

# Step 2: Execute the next step of the task
step_output = agent.run_step(task.task_id)
print(f"Step 2 Output: {step_output.output.response}")


Step-Wise Task Execution:
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Step 1 Output: Observation: 363
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of (121 * 3) + 42 is 363.
Step 2 Output: The result of (121 * 3) + 42 is 363.


In [22]:
# Inspect the task's state
print("\nTask Details:")
tasks = agent.list_tasks()
print(f"Total Tasks: {len(tasks)}")

task_state = tasks[-1]
print(f"Task Input: {task_state.input}")


Task Details:
Total Tasks: 3
Task Input: What is (121 * 3) + 42?


In [23]:

# Completed Steps
completed_steps = agent.get_completed_steps(task_state.task_id)
print(f"\nCompleted Steps ({len(completed_steps)}):")
for idx, step in enumerate(completed_steps):
    print(f"Step {idx + 1}:")
    print(f"Response: {step.output.response}")
    print(f"Sources: {step.output.sources}")


Completed Steps (2):
Step 1:
Response: Observation: 363
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363, is_error=False)]
Step 2:
Response: The result of (121 * 3) + 42 is 363.
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363, is_error=False)]
